In [ ]:
#인터파크투어
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import json
import csv
import time

#원래 max는 140228, num=24
max_=140148
num=24

f = open('interpark_1003_Star.csv', 'w', encoding='utf-8', newline='')
w = csv.writer(f)

f2 = open('interpark_1003_Nostar.csv', 'w', encoding='utf-8', newline='')
w2=csv.writer(f2)

while(num<max_):
    #time.sleep(0.01)
    
    print(num)
    url="http://tour.interpark.com/customer/reviewview.aspx?Seq="+str(num)+"&Type=002&PageNum=10839&GoodsType=999&Region=999&Sort=A&keyWord="
    
    try:    
        page=urlopen(url)
        
        document=page.read()
        page.close()
    
        soup=BeautifulSoup(document, 'html.parser')
    except:
        print("error발생, sleep후 재개")
        time.sleep(3)
        continue;#여기서는 while문으로 돌고있으므로 num조절하지 않아도 됨
            
    #삭제된 페이지는 건너뜀
    if str(type(soup.find('p', {'class':'grade-spot'})))=="<class 'NoneType'>":
        num+=1
        #print('삭제된 리뷰\n')
        continue
        
    grade=soup.find('p', {'class':'grade-spot'}).find_all('img')#find_all을 하면 리스트가 만들어짐. [<img>, <img>, <img>]
    index=["상품구성", "상품정보안내", "일정구성"]
    
    sumGrade=0
    for i in range(0, 3):
    #attrs를 하면 <img>에 있는 속성과 값들이 딕셔너리 형태로 저장됨
        get_grade=grade[i].attrs
    
        if get_grade['alt']!="평점점":#평점데이터가 없는게 아니면
        #그래서 키 값으로 검색하는데 alt의 값이 findall로 하니까 리스트 형태로 나와서 0번으로 해서 숫자만 나오게 한거
            ##print(index[i], ":", re.findall(r"\d", get_grade['alt'])[0])
            grade_=int(re.findall(r"\d", get_grade['alt'])[0])
            sumGrade+=grade_
        else:
            """평점 안매겼어"""

    avg=(sumGrade/3)*2
    
    #if avg==0:
    #    num+=1
    #    continue;
                      
    #리뷰를 한줄한줄 하고싶으면 이렇게.
    test2=soup.find('div', {'class':'edit-view editor-view'}).find_all('p')#p대신 font를 써봤다. 하도 에러가 나서.

    strList=[]
        
    for i in range(len(test2)):
        #temp=test2[i].get_text()
        
        #if temp not in strList:
        #    strList.append(temp)
        #    print(temp)
        strList.append(test2[i].get_text())
    
    if strList==[]:
        num+=1
        continue;#리뷰도 없으면 건너뛰어야지..

    
    text=""
    for i in range(len(strList)):
        text=text+strList[i]+'\n'    

    if avg!=0:
        w.writerow([round(avg, 2), text])
    else:
        w2.writerow([round(avg, 2), text])

    num+=1

f.close()
f2.close()